# Notebook 2: Prompt Structure & Constraints

This notebook covers building reusable, controlled prompts:
- Why static prompts are bad
- Parameterized prompt templates
- Prompt composition patterns
- Hard vs soft constraints
- Style and tone control

## Setup

In [1]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

## Part 1: Why Static Prompts Are Bad

### The Problem with Hardcoded Prompts

In [2]:
# ❌ Bad: Static, hardcoded prompt
def generate_product_description_bad():
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": "Write a product description for a blue backpack."}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

print("❌ Static prompt output:")
print(generate_product_description_bad())

❌ Static prompt output:
**Product Description: Blue Horizon Backpack**

Elevate your everyday adventures with the Blue Horizon Backpack – the perfect blend of style, functionality, and comfort. Crafted from durable, water-resistant fabric, this striking blue backpack is designed to withstand the rigors of daily life while keeping your belongings safe and secure.

**Features:**

- **Spacious Design:** With a generous main compartment, the Blue Horizon Backpack easily accommodates books, laptops, and gym gear, making it ideal for school, work, or travel. Additional front pockets provide quick access to smaller items like your phone, wallet, and keys.

- **Comfort in Every Step:** The padded shoulder straps and breathable back panel ensure maximum comfort, allowing you to carry your essentials with ease, whether you're commuting through the city or hiking up a trail.

- **Stylish Aesthetics:** The vibrant blue hue adds a pop of color to any outfit, while the sleek design makes it suitable

**Problems:**
- Can't reuse for different products
- Can't control tone or length
- Can't version or test easily
- Mixes data with instructions

## Part 2: Parameterized Prompt Templates

**Prompts should be functions, not strings.**

In [3]:
# ✅ Good: Parameterized prompt template
def generate_product_description(
    product_name: str,
    color: str,
    features: list[str],
    tone: str = "professional",
    max_words: int = 50
) -> str:
    """Generate a product description with configurable parameters."""
    
    # Build the prompt dynamically
    system_prompt = f"""You are a product copywriter.
Write in a {tone} tone.
Keep descriptions under {max_words} words."""
    
    features_text = ", ".join(features)
    user_prompt = f"""Write a product description for:
Product: {product_name}
Color: {color}
Features: {features_text}"""
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.7
    )
    
    return response.choices[0].message.content

# Test with different parameters
print("✅ Parameterized prompt (professional tone):")
print(generate_product_description(
    product_name="Hiking Backpack",
    color="blue",
    features=["waterproof", "30L capacity", "padded straps"],
    tone="professional",
    max_words=50
))
print()

✅ Parameterized prompt (professional tone):
Discover the ultimate hiking companion with our blue 30L waterproof backpack. Designed for adventure, it features padded straps for comfort and durability. Perfect for carrying your essentials while keeping them dry, this backpack is ideal for both day hikes and extended treks. Embrace the outdoors with confidence.



In [4]:
# Same function, different parameters
print("✅ Parameterized prompt (casual tone):")
print(generate_product_description(
    product_name="Hiking Backpack",
    color="blue",
    features=["waterproof", "30L capacity", "padded straps"],
    tone="casual and enthusiastic",
    max_words=30
))

✅ Parameterized prompt (casual tone):
Hit the trails with our blue hiking backpack! With a waterproof design and 30L capacity, it’s perfect for all your adventures. Plus, padded straps keep you comfy all day long!


**Key Benefits:**
- Reusable across different products
- Testable with different inputs
- Easy to version and maintain
- Separates data from instructions

## Part 3: Prompt Composition

Build complex prompts from smaller, reusable components.

In [5]:
# Reusable prompt components
TONE_INSTRUCTIONS = {
    "professional": "Use formal language. Be clear and concise.",
    "casual": "Use friendly, conversational language. Be enthusiastic.",
    "technical": "Use precise technical terminology. Focus on specifications."
}

FORMAT_INSTRUCTIONS = {
    "paragraph": "Write as a single paragraph.",
    "bullets": "Write as bullet points.",
    "headline": "Write as a short headline followed by 2-3 sentences."
}

def compose_system_prompt(tone: str, format_type: str, max_words: int) -> str:
    """Compose a system prompt from reusable components."""
    return f"""You are a product copywriter.

Tone: {TONE_INSTRUCTIONS.get(tone, TONE_INSTRUCTIONS['professional'])}
Format: {FORMAT_INSTRUCTIONS.get(format_type, FORMAT_INSTRUCTIONS['paragraph'])}
Length: Keep under {max_words} words."""

# Test composition
system_prompt = compose_system_prompt(tone="technical", format_type="bullets", max_words=60)

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Describe a 30L waterproof hiking backpack with padded straps."}
    ],
    temperature=0.7
)

print("🔧 Composed prompt (technical + bullets):")
print(response.choices[0].message.content)

🔧 Composed prompt (technical + bullets):
- **Capacity**: 30 liters, ideal for day hikes or short excursions  
- **Waterproof Material**: High-density nylon with water-repellent coating  
- **Padded Straps**: Ergonomically designed for comfort and weight distribution  
- **Compartments**: Multiple zippered pockets for organization  
- **Ventilation**: Mesh back panel for breathability  
- **Durability**: Reinforced stitching and heavy-duty zippers  


**Key Insight:** Build a library of reusable components. Mix and match as needed.

## Part 4: Hard vs Soft Constraints

### Soft Constraints (Suggestions)
The model *tries* to follow but may deviate.

In [6]:
# Soft constraint: "Try to keep it short"
response_soft = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant. Try to keep responses brief."},
        {"role": "user", "content": "Explain quantum computing."}
    ],
    temperature=0
)

print("💭 Soft constraint (may be long):")
print(response_soft.choices[0].message.content)
print(f"\nWord count: {len(response_soft.choices[0].message.content.split())}")
print()

💭 Soft constraint (may be long):
Quantum computing is a type of computation that uses quantum bits, or qubits, which can represent and process information in ways that classical bits cannot. Unlike classical bits, which are either 0 or 1, qubits can exist in superpositions of states, allowing them to perform multiple calculations simultaneously.

Key concepts in quantum computing include:

1. **Superposition**: Qubits can be in a combination of 0 and 1 states at the same time, enabling parallel processing.

2. **Entanglement**: Qubits can be entangled, meaning the state of one qubit is dependent on the state of another, no matter the distance between them. This property allows for complex correlations and faster information processing.

3. **Quantum Gates**: Operations on qubits are performed using quantum gates, which manipulate their states through unitary transformations.

4. **Quantum Algorithms**: Certain algorithms, like Shor's algorithm for factoring and Grover's algorithm for s

### Hard Constraints (Strict Rules)
Make the constraint explicit and non-negotiable.

In [7]:
# Hard constraint: Exact format requirement
response_hard = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": """You are a helpful assistant.
STRICT RULE: Respond in EXACTLY 2 sentences. No more, no less."""
        },
        {"role": "user", "content": "Explain quantum computing."}
    ],
    temperature=0
)

print("🔒 Hard constraint (exactly 2 sentences):")
print(response_hard.choices[0].message.content)
print(f"\nSentence count: {response_hard.choices[0].message.content.count('.')}")

🔒 Hard constraint (exactly 2 sentences):
Quantum computing is a type of computation that utilizes the principles of quantum mechanics, such as superposition and entanglement, to process information in fundamentally different ways than classical computers. This allows quantum computers to solve certain complex problems much faster than traditional computers, potentially revolutionizing fields like cryptography and optimization.

Sentence count: 2


**Key Insight:** Use explicit language like "STRICT RULE", "ALWAYS", "NEVER", "EXACTLY" for hard constraints.

*(Note: Period count above is an approximate proxy for sentence count—e.g. "U.S." or abbreviations can skew it.)*

## Part 5: Limiting Scope ("Only Use Provided Context")

Prevent hallucinations by constraining the model to specific information.

In [8]:
# Without scope limitation
response_no_limit = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "What are the features of the TechCorp X1 laptop?"}
    ],
    temperature=0
)

print("❌ Without scope limitation (may hallucinate):")
print(response_no_limit.choices[0].message.content)
print()

❌ Without scope limitation (may hallucinate):
As of my last update in October 2023, I don't have specific information about a "TechCorp X1" laptop, as it may not exist or may not have been widely recognized at that time. However, if you are looking for features typically found in high-end laptops, especially those aimed at professionals or power users, I can provide a general list of features you might expect:

1. **Processor**: Latest generation Intel Core i7/i9 or AMD Ryzen 7/9 processors for high performance.

2. **RAM**: Options for 16GB, 32GB, or even 64GB of RAM for multitasking capabilities.

3. **Storage**: Fast SSD options ranging from 512GB to 2TB, with NVMe technology for quicker data access.

4. **Display**: High-resolution display options, such as 4K (3840 x 2160) or QHD (2560 x 1440), with IPS technology for better color accuracy and viewing angles.

5. **Graphics**: Integrated graphics or dedicated GPUs like NVIDIA GeForce RTX series for gaming and graphic-intensive task

In [9]:
# With scope limitation
product_context = """TechCorp X1 Laptop:
- 14-inch display
- 16GB RAM
- 512GB SSD
- Intel i7 processor
- Price: $1,299"""

response_limited = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": """You are a product information assistant.
STRICT RULE: Only provide information from the context provided.
If information is not in the context, say "I don't have that information."
Never make up or infer details."""
        },
        {
            "role": "user",
            "content": f"""Context:
{product_context}

Question: What are the features of the TechCorp X1 laptop?"""
        }
    ],
    temperature=0
)

print("✅ With scope limitation (only uses context):")
print(response_limited.choices[0].message.content)
print()

✅ With scope limitation (only uses context):
The features of the TechCorp X1 laptop are:
- 14-inch display
- 16GB RAM
- 512GB SSD
- Intel i7 processor
- Price: $1,299



In [10]:
# Test with question outside context
response_outside = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": """You are a product information assistant.
STRICT RULE: Only provide information from the context provided.
If information is not in the context, say "I don't have that information."
Never make up or infer details."""
        },
        {
            "role": "user",
            "content": f"""Context:
{product_context}

Question: What is the battery life of the TechCorp X1 laptop?"""
        }
    ],
    temperature=0
)

print("✅ Question outside context (should refuse):")
print(response_outside.choices[0].message.content)

✅ Question outside context (should refuse):
I don't have that information.


**Key Insight:** Explicit scope limitations prevent hallucinations in RAG (Retrieval-Augmented Generation) systems.

## Part 6: Style and Tone Control

Control output style with specific constraints.

In [11]:
def generate_with_style(content: str, style_rules: list[str]) -> str:
    """Generate content with specific style constraints."""
    style_instructions = "\n".join([f"- {rule}" for rule in style_rules])
    
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": f"""You are a content writer.
Follow these style rules strictly:
{style_instructions}"""
            },
            {"role": "user", "content": content}
        ],
        temperature=0.7
    )
    
    return response.choices[0].message.content

# Test different style constraints
print("📝 Style: Technical documentation")
print(generate_with_style(
    "Explain how to install the software.",
    style_rules=[
        "Use numbered steps",
        "Be precise and technical",
        "Include command examples",
        "No marketing language"
    ]
))
print()

📝 Style: Technical documentation
To install the software, follow these numbered steps:

### Step 1: Verify System Requirements
1. Check the minimum system requirements for the software, including:
   - Operating System version
   - Processor type
   - RAM size
   - Disk space

### Step 2: Download the Installer
1. Visit the official website or repository where the software is hosted.
2. Locate the download link for your operating system.
3. Execute the following command in your terminal (for Linux users):
   ```bash
   wget https://example.com/software-installer.tar.gz
   ```

### Step 3: Extract the Installer
1. Navigate to the directory where the installer was downloaded:
   ```bash
   cd ~/Downloads
   ```
2. Extract the downloaded file:
   ```bash
   tar -xvzf software-installer.tar.gz
   ```

### Step 4: Run the Installer
1. Navigate to the extracted directory:
   ```bash
   cd software-installer
   ```
2. Execute the installer script. If it’s a shell script, run:
   ```bash
   ./

In [12]:
print("📝 Style: Marketing copy")
print(generate_with_style(
    "Explain how to install the software.",
    style_rules=[
        "Use enthusiastic language",
        "Focus on benefits, not technical details",
        "Keep it simple and friendly",
        "End with a call to action"
    ]
))

📝 Style: Marketing copy
Absolutely! Installing new software can be an exciting adventure, and I'm here to make it super simple for you. Just follow these easy steps, and you'll be up and running in no time!

1. **Download the Software**: Head over to the official website and grab the latest version. It's like opening a gift!

2. **Locate the File**: Once downloaded, find the file in your downloads folder. It’s usually named something like "setup" or "installer." 

3. **Start the Installation**: Double-click on the file to kick off the installation process. You’re one step closer to enjoying all the amazing features!

4. **Follow the Prompts**: A friendly installation wizard will guide you through the process. Just click "Next" and read the prompts. It’s as easy as following a recipe!

5. **Finish Up**: Once everything is set, click "Finish." Congratulations! You've just installed new software that will make your life easier and more fun!

6. **Launch the Software**: Find the icon on yo

## Part 7: Building a Reusable Prompt System

Putting it all together: a flexible, reusable prompt builder.

<img src="img/n2_img_1.png" alt="Building a Reusable Prompt System" width="560" style="max-width: 100%; height: auto;">

In [13]:
class PromptBuilder:
    """A reusable prompt builder with constraints and templates."""
    
    def __init__(self, model: str = "gpt-4o-mini"):
        self.model = model
        self.system_instructions = []
        self.constraints = []
        self.context = None
    
    def set_role(self, role: str) -> 'PromptBuilder':
        """Set the assistant's role."""
        self.system_instructions.append(f"You are a {role}.")
        return self
    
    def add_constraint(self, constraint: str, strict: bool = False) -> 'PromptBuilder':
        """Add a constraint (hard or soft)."""
        prefix = "STRICT RULE:" if strict else ""
        self.constraints.append(f"{prefix} {constraint}".strip())
        return self
    
    def set_context(self, context: str) -> 'PromptBuilder':
        """Set context for scope limitation."""
        self.context = context
        return self
    
    def build_system_message(self) -> str:
        """Build the complete system message."""
        parts = self.system_instructions.copy()
        
        if self.constraints:
            parts.append("\nConstraints:")
            parts.extend([f"- {c}" for c in self.constraints])
        
        return "\n".join(parts)
    
    def execute(self, user_message: str, temperature: float = 0.7) -> str:
        """Execute the prompt."""
        system_message = self.build_system_message()
        
        # Add context to user message if provided
        if self.context:
            user_message = f"Context:\n{self.context}\n\nQuestion: {user_message}"
        
        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature
        )
        
        return response.choices[0].message.content

# Example usage
prompt = (PromptBuilder()
    .set_role("technical documentation writer")
    .add_constraint("Use numbered steps", strict=True)
    .add_constraint("Keep under 100 words", strict=True)
    .add_constraint("Include code examples where relevant")
)

result = prompt.execute("How do I create a virtual environment in Python?")

print("🏗️ PromptBuilder output:")
print(result)

🏗️ PromptBuilder output:
1. **Open a terminal** or command prompt.
2. **Navigate** to your project directory using `cd path/to/your/project`.
3. **Install virtualenv** (if not already installed):
   ```bash
   pip install virtualenv
   ```
4. **Create the virtual environment**:
   ```bash
   virtualenv venv
   ```
5. **Activate the virtual environment**:
   - On Windows:
     ```bash
     venv\Scripts\activate
     ```
   - On macOS/Linux:
     ```bash
     source venv/bin/activate
     ```
6. **Deactivate** when done:
   ```bash
   deactivate
   ```
